In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, datasets
import os
from PIL import Image
import matplotlib.pyplot as plt
import tensorboard

In [152]:
class ConvolutionalNeuralNet(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  # 3: colors - R G B, 6: output layer size, 5: convolution kernel size
        self.pool = nn.MaxPool2d(4, 4)  #4: Pool size, 4: Stride size
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*23*23, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

        self.model_feature_learning = torch.nn.Sequential(
            self.conv1,
            nn.ReLU(),
            self.pool,
            self.conv2,
            nn.ReLU(),
            self.pool
        )

        self.model_classification = torch.nn.Sequential(
            self.fc1,
            nn.ReLU(),
            self.fc2,
            nn.ReLU(),
            self.fc3
        )

    def forward(self, x):  # n: number samples in a batch.
        # Start with n, 3, 32, 32
        x = self.model_feature_learning(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.model_classification(x)
        return x


# Use local GPU for CNN models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [127]:
# Set ML configuration
image_size = 400
data_dir = rf'../Pneumonia_classification_data/reshape_{image_size}'
num_epochs = 5
batch_size = 4
rate_learning = 0.001
classes = ('Normal', 'Pneumonia')

In [128]:
# Load and transform datasets
# Images are processed from main.py to 300x300 greyscale jpeg format
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')


transform = transforms.Compose(
    [transforms.Grayscale(num_output_channels=1),
     transforms.ToTensor()])
dataset_test = datasets.ImageFolder(test_dir, transform=transform)
dataset_train = datasets.ImageFolder(train_dir, transform=transform)
dataset_val = datasets.ImageFolder(val_dir, transform=transform)

loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

In [153]:
# Setup model, loss, optimizer and total training steps
model = ConvolutionalNeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=rate_learning)
n_total_steps = len(loader_train)

In [155]:
# Start model training
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print('Finished Training')

Epoch [1/5], Step [100/1277], Loss: 0.5527
Epoch [1/5], Step [200/1277], Loss: 0.3094
Epoch [1/5], Step [300/1277], Loss: 0.3218
Epoch [1/5], Step [400/1277], Loss: 0.5589
Epoch [1/5], Step [500/1277], Loss: 0.5720
Epoch [1/5], Step [600/1277], Loss: 0.2847
Epoch [1/5], Step [700/1277], Loss: 0.7660
Epoch [1/5], Step [800/1277], Loss: 0.2731
Epoch [1/5], Step [900/1277], Loss: 0.2760
Epoch [1/5], Step [1000/1277], Loss: 0.7663
Epoch [1/5], Step [1100/1277], Loss: 0.1945
Epoch [1/5], Step [1200/1277], Loss: 0.7668
Epoch [2/5], Step [100/1277], Loss: 0.1398
Epoch [2/5], Step [200/1277], Loss: 0.2694
Epoch [2/5], Step [300/1277], Loss: 0.3978
Epoch [2/5], Step [400/1277], Loss: 0.7461
Epoch [2/5], Step [500/1277], Loss: 0.3830
Epoch [2/5], Step [600/1277], Loss: 0.1134
Epoch [2/5], Step [700/1277], Loss: 0.0509
Epoch [2/5], Step [800/1277], Loss: 1.0136
Epoch [2/5], Step [900/1277], Loss: 0.3464
Epoch [2/5], Step [1000/1277], Loss: 0.0629
Epoch [2/5], Step [1100/1277], Loss: 0.3963
Epoch 

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(2)]
    n_class_samples = [0 for i in range(2)]
    for images, labels in loader_test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predictions == labels).sum().item()

        for i in range(len(labels)):
            label = labels[i]
            pred = predictions[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(2):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc:.2f} %')